In [8]:
# load libraries and scripts
using HDF5
using Trapz
#using Statistics
include("../scripts/plotting.jl")
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/disturbances.jl")
include("../scripts/stable.jl")
include("../scripts/dynamics.jl")

vectorize (generic function with 1 method)

In [9]:
#load borders
border, scale_factor = import_border("../data/borders/border.json");

data, ~ = import_json_numerics("../data/test/triangle.json");
border = data["border"]

border


4×2 Matrix{Float64}:
  0.0  -0.866025
  1.0   0.866025
 -1.0   0.866025
  0.0  -0.866025

In [10]:
# create the lattice grid
include("../scripts/get_grid.jl")
dx = 0.01
@time begin
Nx, Ny, xrange, yrange, isinside, isborder, n, isgrid = get_grid(border, Float64(dx));
end

#@time begin
#Nx, Ny, xrange, yrange, isinside, isborder, n, isgrid = get_grid_fast(border, Float64(dx));
#end

  0.005628 seconds (146.94 k allocations: 16.408 MiB)


In [40]:
#p = [(xrange[j],yrange[i]) for i in 1:Ny for j in 1:Nx]
p = [(yrange[i], xrange[j]) for i in 1:Ny for j in 1:Nx]
x = repeat(reshape(xrange,1,Nx), Ny, 1)
y = repeat(reshape(yrange,Ny,1), 1, Nx);
#x = repeat(xrange,1,1:Ny)
#println(size(y))
#println(size(x))
println(size(p))
println(size(vec(isinside)))
scatter([p[i][1] for i in 1:length(p)], [p[i][2]  for i in 1:length(p)])

(36312,)
(36312,)


LoadError: syntax: missing separator in array expression

In [ ]:
m = 30 * ones(Ny, Nx)
d = 0.6 * m
bx = 10 * ones(Ny, Nx)
by = 10 * ones(Ny, Nx)
p = zeros(Ny, Nx)
m[.!isgrid] .= 0
d[.!isgrid] .= 0
p[.!isgrid] .= 0;

In [ ]:
@time begin
isinsideflat, pflat, minvflat, gammaflat, xi = vectorize(isinside, isborder, n, bx, by, p, m, d);
end

#@time begin
#isinsideflat, pflat, minvflat, gammaflat, xi2 = vectorize(isinside, isborder, n, bx, by, p, m, d);
#end
#sum(abs.(Array{Float64,2}(xi)-Array{Float64,2}(xi2)))

In [ ]:
isgridflat = vec(isinside .| isborder);

In [ ]:
# define a disturbance
dP = -9.0
# dP = 0.0
sigma = 100.0 / scale_factor
location = [-1500.0, -900.0] / scale_factor
sigma = 0.1
location = [00.0, 00.0] / scale_factor
dp = local_disturbance(isgrid, xrange, yrange, location, dP, sigma)
dpflat = vec(dp)
println("Synchronized frequency: ", trapz((yrange, xrange), p .+ dp) / trapz((yrange, xrange), d))
println("Synchronized frequency: ", sum(p .+ dp) / sum(d))
hm_plot(isinside, xrange, yrange, dp)


In [ ]:
th0 = zeros(Ny * Nx)

ts, ~, omegas = perform_dyn_sim(isgridflat, xi, pflat[isgridflat]+dpflat[isgridflat],
    minvflat, gammaflat, th0, interval = 100, Ndt = Int64(30000), dt = 0.001, method = "crank-nicolson")
o_ref = sum(p .+ dp) / sum(d)
sum(omegas[:,end])/sum(isgridflat) / o_ref

In [ ]:
# backward Euler
coord = [500 -250; 0.0 0.0; -500 500; 500. 500.; -1500.0 -1000.0; 750.0 -1000; -1000 -400]/scale_factor
time_plot(ts_be, omegas_be, coord, tstart = 0.0, tend = 50.0, borders = [border])

In [ ]:
# Crank-Nicolson
coord = [500 -250; 0.0 0.0; -500 500; 500. 500.; -1500.0 -1000.0; 750.0 -1000; -1000 -400]
time_plot(ts_cn, omegas_cn, coord, tstart = 0.0, tend = 10.0, borders = [border])

In [ ]:
# "Standard vec"
coord = [500 -250; 0.0 0.0; -500 500; 500. 500.; -1500.0 -1000.0; 750.0 -1000; -1000 -400]
time_plot(ts_v, omegas_v, coord, tstart = 0.0, tend = 10.0, borders = [border])

In [ ]:
heatmap(omegas_be[:,:,10],fill=true)


In [ ]:
N = sum(isgridflat)
dt = 0.001
th0 = zeros(Ny * Nx)

I = sparse(1:N, 1:N, ones(N))
A = [I -dt / 2 * I;
    - dt / 2 / dx^2 * sparse(1:N, 1:N, minvflat) * xi (I + dt/2 * sparse(1:N, 1:N, gammaflat))]
B = [I dt / 2 * I;
    dt / 2 / dx^2 * sparse(1:N, 1:N, minvflat) * xi (I - dt/2 * sparse(1:N, 1:N, gammaflat))]
C = [zeros(N); dt * sparse(1:N, 1:N, minvflat) * dpflat[isgridflat]]
x = [zeros(N); th0[isgridflat]]
x2 = [zeros(N); th0[isgridflat]]
println(size(x))
println(size(A))
println(size(B))
println(size(C))
@time begin
    x1 = A \ (B * x + C)
end
x = [zeros(N); th0[isgridflat]]
@time begin
    gmres!(x2, A, (B * x + C))
end

println(sum(abs.(A * x1 - (B * x + C))))
println(sum(abs.(A * x2 - (B * x + C))))